# Лабораторная работа №2
## Выполнил студент группы БВТ2003 Митрофанов Алексей Олегович


## Задание №1
Реализовать методы поиска: бинарный, бинарное дерво, фибоначчиев, интерполяционный. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы страндартной функции поиска.

In [56]:
from random import randint

array = sorted([randint(0, 1000000) for i in range(1990)])
searched_value = array[1368]


In [57]:
def binarySearch(
    array: list[int], element: int, start: int = 0, end: int = len(array) - 1
):
    if start > end:
        return -1

    mid = (start + end) // 2

    if array[mid] == element:
        return mid

    if array[mid] > element:
        return binarySearch(array, element, start, mid - 1)
    elif array[mid] < element:
        return binarySearch(array, element, mid + 1, end)

In [58]:
def interpolationSearch(
    array: list[int], element: int, start: int = 0, end: int = len(array) - 1
):
    if start > end:
        return -1

    mid = start + ((element - array[start]) * (end - start)) // (
        array[end] - array[start]
    )

    if array[mid] == element:
        return mid
    if array[mid] > element:
        return interpolationSearch(array, element, start, mid - 1)
    if array[mid] < element:
        return interpolationSearch(array, element, mid + 1, end)

In [59]:
def fibonacciSearch(array: list[int], element: int):
    offset = 0
    fibMm2 = 0
    fibMm1 = 1
    fibM = fibMm2 + fibMm1

    while fibM < len(array):
        fibMm2 = fibMm1
        fibMm1 = fibM
        fibM = fibMm2 + fibMm1

    while fibM >= 0:
        index = fibMm2 + offset
        if index < len(array) and array[index] == element:
            return fibMm2 + offset
        if index >= len(array) or array[index] < element:
            if index < len(array):  # do NOT cange offset when edge case (right corner)
                offset = offset + fibMm2
            fibM = fibMm1
            fibMm1 = fibMm2
            fibMm2 = fibM - fibMm1
            continue
        if array[fibMm2 + offset] > element:
            if fibMm1 == fibMm2:  # edge case (left corner)
                fibM = 1
                fibMm1 = 1
                fibMm2 = 0
            else:
                fibMm1 = fibMm1 - fibMm2
                fibMm2 = fibMm2 - fibMm1
                fibM = fibMm1 + fibMm2
            continue
    return -1

In [60]:
class BTSNode:
    def __init__(self, value: int, index: int | set[int] = None) -> None:
        self.left = None
        self.right = None
        self.value = value
        if type(index) is set:
            self.index = index
        elif type(index) is int:
            self.index = {index}

    def insert(self, other):
        if self == other:
            self.index.update(other.index)
            return
        if other < self:
            if self.left is not None:
                self.left.insert(other)
                return
            self.left = other
        else:
            if self.right is not None:
                self.right.insert(other)
                return
            self.right = other

    def __eq__(self, other):
        if type(other) is BTSNode:
            return self.value == other.value
        return False

    def __lt__(self, other):
        if type(other) is BTSNode:
            return self.value < other.value
        return False

    def __gt__(self, other):
        if type(other) is BTSNode:
            return self.value > other.value
        return False

    def search(self, other):
        if self == other:
            return self
        if self.right is not None and other > self:
            return self.right.search(other)
        if self.left is not None and other < self:
            return self.left.search(other)
        return None


class BTS:
    def __init__(self, array: list[int | float]) -> None:
        self.array = sorted(array)
        self.middle = len(self.array) // 2
        self.tree = BTSNode(self.array[self.middle], self.middle)

        for index, element in enumerate(self.array):
            self.tree.insert(BTSNode(element, index))

    def search(self, value: int | float):
        search_result = self.tree.search(BTSNode(value, 0))
        if search_result is not None:
            return search_result.index
        return None

## Тестирование

In [61]:
from time import perf_counter_ns as timer

results = {}

In [62]:
wholeTime = 0
tries = 0
for i in range(500000):
    start = timer()
    binarySearch(array, searched_value)
    end = timer()
    tries += 1
    wholeTime += end - start
avgTime = wholeTime/tries
results.update(dict(Бинарный=avgTime))
print(f"Time: {avgTime}")
print(f"Result: {binarySearch(array, searched_value)}")

Time: 2580.982
Result: 1368


In [63]:
wholeTime = 0
tries = 0
for i in range(500000):
    start = timer()
    fibonacciSearch(array, searched_value)
    end = timer()
    tries += 1
    wholeTime += end - start
avgTime = wholeTime/tries
results.update(dict(Фиббоначи=avgTime))
print(f"Time: {avgTime}")
print(f"Result: {fibonacciSearch(array, searched_value)}")

Time: 5141.5482
Result: 1368


In [64]:
wholeTime = 0
tries = 0
for i in range(500000):
    start = timer()
    interpolationSearch(array, searched_value)
    end = timer()
    tries += 1
    wholeTime += end - start
avgTime = wholeTime/tries
results.update(dict(Интерпол=avgTime))
print(f"Time: {avgTime}")
print(f"Result: {interpolationSearch(array, searched_value)}")

Time: 557.25
Result: 1368


In [65]:
bts = BTS(array)

wholeTime = 0
tries = 0
for i in range(5000):
    start = timer()
    bts.search(searched_value)
    end = timer()
    tries += 1
    wholeTime += end - start
avgTime = wholeTime/tries
results.update(dict(Дерево=avgTime))
print(f"Time: {avgTime}")
print(f"Result: {interpolationSearch(array, searched_value)}")

Time: 160413.68
Result: 1368


In [66]:
wholeTime = 0
tries = 0
for i in range(100000):
    start = timer()
    array.index(searched_value)
    end = timer()
    tries += 1
    wholeTime += end - start
avgTime = wholeTime/tries
results.update(dict(Стандарт=avgTime))
print(f"Time: {avgTime}")
print(f"Result: {interpolationSearch(array, searched_value)}")

Time: 15376.867
Result: 1368


In [67]:
leaderBoard = {k: v for k, v in sorted(results.items(), key=lambda item: item[1])}
for i, j in leaderBoard.items():
    print(f"{i}: \t{j} ms")

Интерпол: 	557.25 ms
Бинарный: 	2580.982 ms
Фиббоначи: 	5141.5482 ms
Стандарт: 	15376.867 ms
Дерево: 	160413.68 ms


## Вывод
По результатам тестирования, чаще всего по времени выигрывает интерполяционный поиск, он обгоняет бинарный в 2-5 раз, и только в худшем случае сравнивается с ним.

## Задание 2

Простое рехэширование || С помощью псевдослучайных чисел || Метод цепочек

In [68]:
import random
import sys

In [1]:
class HashTableSimple:
    def __init__(self, size):
        self.size = size
        self.hash_table = self.create_buckets()

    def create_buckets(self):
        return [[] for _ in range(self.size)]

    def rehash_index(self, collision_ind):
        new_ind = collision_ind + 1
        new_ind %= self.size
        return new_ind

    def get_hash(self, key):
        # hashed_bytes = hashlib.sha256(str.encode(key)).digest()
        # hashed_int = int.from_bytes(hashed_bytes, byteorder='big', signed=False)
        hashed_int = ord(key[0])
        hashed_key = hashed_int % self.size
        return hashed_key

    def set_val(self, key, val):        
        hashed_key = self.get_hash(key)

        bucket = self.hash_table[hashed_key]

        if len(bucket) == 0:
            bucket.append((key, val))
            return
        for index, record in enumerate(bucket):
            record_key, record_val = record
        if record_key == key:
                bucket.pop()
                bucket.append((key, val))
                return
        else:
            for _ in range(self.size - 1):
                hashed_key = self.rehash_index(hashed_key)
                bucket = self.hash_table[hashed_key]
                if len(bucket) == 0:
                    bucket.append((key, val))
                    return
                for index, record in enumerate(bucket):
                    record_key, record_val = record
                    if record_key == key:
                        bucket.pop()
                        bucket.append((key, val))
                        return

    def get_val(self, key):
        hashed_key = self.get_hash(key)

        for _ in range(self.size - 1):
            bucket = self.hash_table[hashed_key]

            found_key = False
            for index, record in enumerate(bucket):
                record_key, record_val = record

                if record_key == key:
                    found_key = True
                    break

            if found_key:
                return record_val
            else:
                hashed_key = self.rehash_index(hashed_key)
        return "No record found"

    def delete_val(self, key):
        hashed_key = self.get_hash(key)

        for _ in range(self.size - 1):
            bucket = self.hash_table[hashed_key]

            found_key = False
            for index, record in enumerate(bucket[:]):
                record_key, record_val = record

                if record_key == key:
                    found_key = True
                    break
                
            if found_key:
                bucket.pop(index)
            else:
                hashed_key = self.rehash_index(hashed_key)
        return

    def __str__(self):
        return "\n".join(str(item) for item in self.hash_table)

In [70]:
class HashTablePseudoRandom(HashTableSimple):
    def rehash_index(self, collision_ind):
        random.seed(collision_ind)
        new_ind = collision_ind + random.randint(1, sys.maxsize)
        new_ind %= self.size
        return new_ind

In [71]:
class HashTableChaining(HashTableSimple):
    def set_val(self, key, val):
        hashed_key = self.get_hash(key)

        bucket = self.hash_table[hashed_key]

        for index, record in enumerate(bucket):
            record_key, record_val = record
            if record_key == key:
                bucket[index] = (key, val)
                return

        bucket.append((key, val))

    def get_val(self, key):
        hashed_key = self.get_hash(key)

        bucket = self.hash_table[hashed_key]

        for index, record in enumerate(bucket):
            record_key, record_val = record
            if record_key == key:
                return record_val

    def delete_val(self, key):
        hashed_key = self.get_hash(key)

        bucket = self.hash_table[hashed_key]

        for index, record in enumerate(bucket[:]):
            record_key, record_val = record
            if record_key == key:
                del bucket[index]

In [72]:
table_size = 10

In [83]:
hash_table = HashTableSimple(table_size)

hash_table.set_val("a", "va")
hash_table.set_val("ab", "vab")
hash_table.set_val("abc", "vabc")
hash_table.set_val("abcd", "vabcd")
hash_table.set_val("bcd", "vbcd")
hash_table.set_val("ab", "vAB")



print(hash_table)
print("'abc' -> " + hash_table.get_val("abc"))
print()

[('abcd', 'vabcd')]
[('bcd', 'vbcd')]
[]
[]
[]
[]
[]
[('a', 'va')]
[('ab', 'vAB')]
[('abc', 'vabc')]
'abc' -> vabc



In [74]:
hash_table = HashTablePseudoRandom(table_size)

hash_table.set_val("a", "va")
hash_table.set_val("ab", "vab")
hash_table.set_val("abc", "vabc")
hash_table.set_val("abcd", "vabcd")
hash_table.set_val("bcd", "vbcd")
hash_table.set_val("ab", "vAB")
hash_table.set_val("abcde", "vabcde")



print(hash_table)
print("'abc' -> " + hash_table.get_val("abc"))
print()

[('abcd', 'vabcd')]
[]
[('bcd', 'vbcd')]
[]
[('abc', 'vabc')]
[]
[]
[('a', 'va')]
[('ab', 'vAB')]
[]
'abc' -> vabc



In [75]:
hash_table = HashTableChaining(table_size)

hash_table.set_val("a", "va")
hash_table.set_val("ab", "vab")
hash_table.set_val("abc", "vabc")
hash_table.set_val("abcd", "vabcd")
hash_table.set_val("bcd", "vbcd")
hash_table.set_val("ab", "vAB")
hash_table.set_val("abcde", "vabcde")



print(hash_table)
print("'abc' -> " + hash_table.get_val("abc"))
print()

[]
[]
[]
[]
[]
[]
[]
[('a', 'va'), ('ab', 'vAB'), ('abc', 'vabc'), ('abcd', 'vabcd'), ('abcde', 'vabcde')]
[('bcd', 'vbcd')]
[]
'abc' -> vabc



## Задание 3

Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям.  
Написать программу,  которая находит хотя бы один способ решения задач.


In [76]:
found_eight_queens = []

In [77]:
def is_diagonal_vacant(current_found_queens, x, y):
    for position in current_found_queens:
        xf, yf = position
        if (abs(xf-x) - abs(yf-y) == 0):
            return False
    return True

In [78]:
def place_queens_reursively(current_found_queens, queen_counter):
    found_queens_x = [position[0] for position in current_found_queens]
    found_queens_y = [position[1] for position in current_found_queens]

    if queen_counter < 8:

        for y in range(8):
            if y not in found_queens_y:

                for x in range(8):
                    if x not in found_queens_x:
                        
                        if is_diagonal_vacant(current_found_queens, x, y):
                            current_found_queens_copy = current_found_queens.copy()
                            current_found_queens_copy.append((x, y))
                            place_queens_reursively(current_found_queens_copy[:], queen_counter + 1)
    else:
        if len(found_eight_queens) > 0:
            if set(current_found_queens) not in found_eight_queens:
                found_eight_queens.append(set(current_found_queens))
        else:
            found_eight_queens.append(set(current_found_queens))

In [79]:
def count_qeens_for_cords(x, y):
    current_queens = [(x, y)]
    return place_queens_reursively(current_queens, 1)

In [80]:
for i in range(8):
    count_qeens_for_cords(i, 0)

In [81]:
print("Solutions found: " + str(len(found_eight_queens)))

Solutions found: 92


In [82]:
clear_board = [[0 for i in range(8)] for i in range(8)]

def print_board(board):
    for i in range(8):
        # print(board[i])
        formatted = []
        for position in board[i]:
            if position == 1:
                formatted.append('■')
            else:
                formatted.append('□')
        print('  '.join(formatted))

solution_counter = 1
for solution in found_eight_queens:
    board = [[0 for i in range(8)] for i in range(8)]
    for position in solution:
        x, y = position
        board[x][y] = 1
    print()
    print(f'Solution #{solution_counter}')
    print_board(board)
    solution_counter += 1


Solution #1
■  □  □  □  □  □  □  □
□  □  □  □  □  □  ■  □
□  □  □  □  ■  □  □  □
□  □  □  □  □  □  □  ■
□  ■  □  □  □  □  □  □
□  □  □  ■  □  □  □  □
□  □  □  □  □  ■  □  □
□  □  ■  □  □  □  □  □

Solution #2
■  □  □  □  □  □  □  □
□  □  □  □  □  □  ■  □
□  □  □  ■  □  □  □  □
□  □  □  □  □  ■  □  □
□  □  □  □  □  □  □  ■
□  ■  □  □  □  □  □  □
□  □  □  □  ■  □  □  □
□  □  ■  □  □  □  □  □

Solution #3
■  □  □  □  □  □  □  □
□  □  □  □  □  ■  □  □
□  □  □  □  □  □  □  ■
□  □  ■  □  □  □  □  □
□  □  □  □  □  □  ■  □
□  □  □  ■  □  □  □  □
□  ■  □  □  □  □  □  □
□  □  □  □  ■  □  □  □

Solution #4
■  □  □  □  □  □  □  □
□  □  □  □  ■  □  □  □
□  □  □  □  □  □  □  ■
□  □  □  □  □  ■  □  □
□  □  ■  □  □  □  □  □
□  □  □  □  □  □  ■  □
□  ■  □  □  □  □  □  □
□  □  □  ■  □  □  □  □

Solution #5
□  □  □  □  □  ■  □  □
■  □  □  □  □  □  □  □
□  □  □  □  ■  □  □  □
□  ■  □  □  □  □  □  □
□  □  □  □  □  □  □  ■
□  □  ■  □  □  □  □  □
□  □  □  □  □  □  ■  □
□  □  □  ■  □  □  □  □

Solution #6
□ 